In [1]:
import numpy as np
import pandas as pd
import json
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

In [2]:
# Load "DataSchools" files.

DataSchools = pd.read_csv("../3_Data_Munging/csv_files/DataSchools.csv")

In [3]:
# Load GeoJSON information.

with open("geo_files/geojson_districts") as data_file:
    geojson_districts = json.load(data_file)

with open("geo_files/geojson_schools") as data_file:
    geojson_schools = json.load(data_file)

In [ ]:
# Method 1 (don't use "updatemenus")

app = dash.Dash()

mapbox_access_token = 'pk.eyJ1IjoiZ2lsdHJhcG8iLCJhIjoiY2o4eWJyNzY4MXQ1ZDJ3b2JsZHZxb3N0ciJ9.MROnmydnXtfjqjIBtC-P5g'

app.layout = html.Div([
    html.Div(["Schoolarize"    
    ], style = {
        "height": "10%",
        "background-color": "blue"}),
    html.Div([
        dcc.Graph(id = "map", style = {"height": "100%"})
    ], style = {
        "width": "80%",
        "height": "90%",
        "vertical-align": "top",
        "display": "inline-block",
        "background-color": "rgb(255,0,255)"}),
    html.Div([
        dcc.Dropdown(
            id = "dist-drop",
            options = [{"label": str(district), "value": str(district)}\
                       for district in DataSchools["Distrito"].unique()],
            placeholder = "Selecciona tu distrito"
        )
    ], style = {
        "width": "20%",
        "height": "90%",
        "vertical-align": "top",
        "display": "inline-block",
        "background-color": "#00ff00"})
], style = {
    "height": "98vh"})

@app.callback(
    Output("map", "figure"),
    [Input("dist-drop", "value")])
def update_graph(value):
    
    if value == "HORTALEZA":
        latmap = 40.48
        lonmap = -3.640482
        zoomap = 12.5
        layers = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if i["properties"]["NOMBRE"] == "Hortaleza"],
                    type = geojson_districts["type"]),
                color = "#eef5db",
                opacity = 0.8
            )
        ]
    elif value == "PUENTE DE VALLECAS":
        latmap = 40.384
        lonmap = -3.669059
        zoomap = 13
        layers = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if i["properties"]["NOMBRE"] == "Puente de Vallecas"],
                    type = geojson_districts["type"]),
                color = "#c7efcf",
                opacity = 0.8
            )
        ]
    elif value == "VILLA DE VALLECAS":
        latmap = 40.352
        lonmap = -3.62135
        zoomap = 12.2
        layers = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = dict(
                    crs = geojson_districts["crs"],
                    features = [i for i in geojson_districts["features"]\
                                if i["properties"]["NOMBRE"] == "Villa de Vallecas"],
                    type = geojson_districts["type"]),
                color = "#c7efcf",
                opacity = 0.8
            )
        ]
    else:
        latmap = 40.48
        lonmap = -3.65
        zoomap = 10
        layers = [
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = geojson_sources["source_1"],
                color = "#eef5db",
                opacity = 0.8
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = geojson_sources["source_2"],
                color = "#c7efcf",
                opacity = 0.8
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = geojson_sources["source_3"],
                color = "#d6d1b1",
                opacity = 0.8
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = geojson_sources["source_4"],
                color = "#f0b67f",
                opacity = 0.8
            ),
            dict(
                type = "fill",
                sourcetype = "geojson",
                source = geojson_sources["source_8"],
                color = "#fe5f55",
                opacity = 0.8
            )
        ]
    
    return dict(
        data = [
            dict(
                type = "scattermapbox",
                lat = DataSchools["Latitud"],
                lon = DataSchools["Longitud"],
                mode = "markers",
                marker = dict(size = 8),
                text = DataSchools["Colegio"]
            )
        ],
        layout = dict(
            autosize = True,
            hovermode = "closest",
            margin = dict(l = 0, r = 0, t = 0, b = 0),
            mapbox = dict(
                accesstoken = mapbox_access_token,
                bearing = 0,
                center = dict(lat = latmap, lon = lonmap),
                pitch = 0,
                zoom = zoomap,
                style = "light",
                layers = layers
            )
        )
    )
    


app.run_server()